In [10]:
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import Html2TextTransformer
import nest_asyncio

nest_asyncio.apply()
urls = ["https://sou.zhaopin.com/?jl=530&kw=%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E5%A4%84%E7%90%86"]
loader = AsyncChromiumLoader(urls,user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
docs = loader.load()
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)
print(docs_transformed[0].page_content)

  * 首页 
  * 北京站 
  * 政企招聘 
  * 校园招聘 
  * 高端职位 
  * 海外招聘 
  * 测评及培训 
  * 职Q社区 

消息 _|_ 我要招人 登录/注册

__ 职位类别 __

  * 不限
  * 销售/商务拓展
  * 人事/行政/财务/法务
  * 互联网/通信及硬件
  * 运维/测试
  * 视觉/交互/设计
  * 运营/专业分析
  * 产品/项目/高级管理
  * 市场/品牌/公关
  * 金融/保险
  * 房地产/工程/建筑
  * 物流/采购/供应链
  * 汽车
  * 生产制造/营运管理
  * 农业/能源/环保
  * 医疗/医美/医务
  * 教育/培训/科研
  * 编辑/记者/翻译
  * 直播/影视/传媒
  * 商务服务/生活服务
  * 管培生/非企业从业者

公司行业 __

  * 不限
  * 互联网/IT/电子/通信
  * 房地产/建筑
  * 金融业
  * 教育培训/科研
  * 广告/传媒/文化/体育
  * 生物医药/医疗
  * 批发/零售/贸易
  * 制造业
  * 汽车
  * 交通运输/仓储/物流
  * 专业服务
  * 生活服务
  * 能源/环保/矿产
  * 政府/非盈利机构
  * 农/林/牧/渔

__

北京

行政区

地铁沿线

薪资要求

  * 不限
  * 4K以下
  * 4K-6K
  * 6K-8K
  * 8K-10K
  * 10K-15K
  * 15K-25K
  * 25K-35K
  * 35K-50K
  * 50K以上

学历要求

  * 不限
  * 初中及以下
  * 高中
  * 中专/中技
  * 大专
  * 本科
  * 硕士
  * MBA/EMBA
  * 博士

工作经验

  * 不限
  * 无经验
  * 1年以下
  * 1-3年
  * 3-5年
  * 5-10年
  * 10年以上

职位类型

  * 不限
  * 全职
  * 兼职/临时
  * 实习
  * 校园

公司性质

  * 不限
  * 国企
  * 外企
  * 合资
  * 民营
  * 上市公司
  * 股份制企业
  * 事业单位
  * 其他

公司规模

  * 不限
  * 20人以下
  * 2

In [12]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "false"

from langchain_core.output_parsers import StrOutputParser
from langchain_fireworks import ChatFireworks
from langchain_core.prompts import PromptTemplate,format_document


chat = ChatFireworks(model="accounts/fireworks/models/mixtral-8x7b-instruct",temperature=0)

urls = ["https://sou.zhaopin.com/?jl=530&kw=%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E5%A4%84%E7%90%86"]
loader = AsyncChromiumLoader(urls,user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
docs = loader.load()
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

doc_prompt = PromptTemplate.from_template("{page_content}")
chain = {
    "content":lambda docs: "\n\n".join(format_document(doc,doc_prompt) for doc in docs_transformed)
} | \
PromptTemplate.from_template("""
提取出下边文本中所有职位信息，每条职位信息应包括: 职位名、薪资范围、公司名、职位地点、HR名字、公司性质。

文本如下: 
                             
{content}""") |\
chat | StrOutputParser()

print(chain.invoke(docs_transformed))

1. 职位名: nlp自然语言处理工程师
薪资范围: 1万-2万
公司名: 四川智服人力资源有限公司
职位地点: 北京·海淀·西北旺
HR名字: 罗镇坤
公司性质: 民营

2. 职位名: 自然语言处理工程师
薪资范围: 2.9万-3.5万
公司名: 北京友安丰廷创新科技有限公司
职位地点: 北京·西城·展览路
HR名字: 宋女士
公司性质: 民营

3. 职位名: 自然语言处理算法工程师
薪资范围: 1.5万-2.5万
公司名: 四川智服人力资源有限公司
职位地点: 北京·海淀·西北旺
HR名字: 张先生
公司性质: 民营

4. 职位名: AI中心算法类实习生-自然语言处理方向/网络大模型项目
薪资范围: 150-200/天
公司名: 中国移动研究院
职位地点: 北京·西城·广安门内
HR名字: 王女士
公司性质: 国企

5. 职位名: 自然语言处理算法工程师
薪资范围: 1.5万-2.5万·13薪
公司名: 北京博润新源科技有限公司
职位地点: 北京·昌平·小汤山
HR名字: 纪女士
公司性质: 民营

6. 职位名: 自然语言处理工程师（项目部）
薪资范围: 1.3万-2.6万
公司名: 北京(海淀)大数据先进技术研究院(北京大数据先进技术研究院)
职位地点: 北京·海淀·四季青
HR名字: 刘雅文
公司性质: 事业单位

7. 职位名: nlp自然语言处理算法工程师
薪资范围: 1.3万-2.6万
公司名: 软通动力信息技术(集团)股份有限公司
职位地点: 北京·海淀·苏家坨
HR名字: 周女士
公司性质: 民营

8. 职位名: nlp自然语言处理研究员
薪资范围: 1.8万-3万
公司名: 北京华夏大地远程教育网络服务有限公司
职位地点: 北京·西城·德胜
HR名字: 夏女士
公司性质: 上市公司

9. 职位名: nlp自然语言处理工程师
薪资范围: 1.2万-2万
公司名: 北京市科学技术研究院
职位地点: 北京·海淀·西北旺
HR名字: 罗丹
公司性质: 事业单位

10. 职位名: 高级自然语言处理算法工程师(J10278)
薪资范围: 2.5万-3.5万
公司名: 北京智慧星光信息技术股份有限公司
职位地点: 北京·海淀·海淀街道
HR名字: 刘女士
公司性质: 民营

11. 职位名: 高级自然语言算法处理工程师
薪资范围: 2.5万-2